<font color='black'>
<h1>
<span style="font-family:verdana; font-size:1.4em;">
Basic Transformations
</span>
</h1>
</font>

<font color='steelblue'>
<h2>
<span style="font-family:verdana; font-size:1.4em;">
Basic Transformation Examples & Exercises<br>
</span>
</h2>
</font>

<font color='grey'>
<span style="font-family:verdana; font-size:1.2em;">
<b>
Transformations on data is one of the most common ETL tasks. Transforming data can range in complexity from simply parsing relevant fields to handle null values without affecting downstream operations.<br>
Common transformations include:
    <ul>
        <li>Imputing null or missing data</li>
        <li>Normalizing values</li>
        <li>Remove Duplicate data</li>
        <li>Exploding arrays</li>
        <li>Pivoting DataFrames</li>
        </ul>
    </b>
</span>
<span style="font-family:verdana; font-size:1.0em;"><br>
    <b>Following is included in this notebook</b>:
    <ul>
        <li>Normalizing Data using user defined function and using MinMaxScaler</li>
        <li>Handling Missing Data by dropping rows, replace missing values with placeholder values and using Imputer to replace missing values</li>
        <li>Dropping duplicate records</li>
        <li>Using Pivot to tranpose a column in dataframe</li>
        <li>Exercise 1: Drop duplicate rows from a csv file</li>
    </ul>
</span>
</font>

<font color='tomato'>
<span style="font-family:verdana; font-size:1.2em;">
    <b>IF RUNNING IN DATABRICK environment, set the next cell to True</b>:
</span>
</font>

In [ ]:
DATA_BRICKS = False

In [ ]:
if DATA_BRICKS == False:
    # Set up the environment for using pyspark
    import findspark
    findspark.init()
    
    from pyspark.sql import SparkSession
    from pyspark import SparkConf, SparkContext
    
    spark = SparkSession.builder.appName("Basic Transformations").getOrCreate()
    sc = spark.sparkContext
    sc.setLogLevel("Error")

In [ ]:
# Data type definitions, # Lot of functions used in this notebook
from pyspark.sql.types import *
from pyspark.sql.functions import *

## Utility Functions

In [ ]:
def showDataFrame(df, numrecs = 5):
    if DATA_BRICKS == False:
        df.limit(numrecs).show(truncate = False)
    else:
        display(df)

<font color='gray'>
<h3>
<span style="font-family:verdana; font-size:1.2em;">
Normalizing Data<br>
</span>
</h3>
</font>
<span style="font-family:verdana; font-size:1.0em;">
Normalizing is where the data is transformed or scaled to some normal form, it could be between small specified range e.g. bounding a range of integers to 0 and 1.<br>
Explore 2 examples of normalizing data:
    <ul>
        <li><b>User Defined: </b> write our own normalizer</li>
        <li><b>Use ML feature library: </b> Use the MinMaxScaler from the machine learning feature library</li>
    </ul>
</span>

In [ ]:
df = spark.range(1000.0, 10000.0)
df = df.withColumn("id", col("id").cast(IntegerType()))

In [ ]:
showDataFrame(df)

In [ ]:
# print number of rows in dataframe
df.count()

## User defined

In [ ]:
from pyspark.sql.functions import col, max, min

colMin = df.select(min("id")).first()[0]
colMax = df.select(max("id")).first()[0]

# print the min and max values


In [ ]:
normalizedDF = (df
                .withColumn("normalizedValue", (col("id") - colMin) / (colMax - colMin) ))

In [ ]:
showDataFrame(normalizedDF)

## Using MinMaxScaler<br>
<span style="font-family:verdana; font-size:1.0em;">
Making use of MinMaxScaler from the pyspark library for machine learning. Because we have only one column in the dataset, it has to be transformed into a vector to be used by the scaler. 
</span>

In [ ]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=["id"],outputCol="features")

In [ ]:
# Now let us use the transform method to transform our dataset
sdf = assembler.transform(df)
sdf.select(["id", "features"]).show(truncate=False)

In [ ]:
from pyspark.ml.feature import MinMaxScaler

# instantiate the class
minMaxScaler = MinMaxScaler(inputCol = "features", outputCol = "minMax")

# set the minimum and maximum values
minMaxScaler.setMin(0)
minMaxScaler.setMax(1)

# train the scaler by doing fit
model = minMaxScaler.fit(sdf)

# transform into new dataframe using the trained model
minmaxDF = model.transform(sdf)

In [ ]:
showDataFrame(minmaxDF)

<font color='gray'>
<h3>
<span style="font-family:verdana; font-size:1.2em;">
Imputing Null or Missing Data<br>
</span>
</h3>
</font>
<span style="font-family:verdana; font-size:1.0em;">
Null or missing data are many times a norm in many datasets. There can be many ways of handling such data.<br>
Here are some examples of dealing with such data:
    <ul>
        <li><b>Dropping records: </b> Works well when such data might not be used in downstream processing</li>
        <li><b>Assign a placehoder value (e.g. -1): </b> Will allow us to examine such data later</li>
        <li><b>Imputing values: </b> Replace the missing values with a mean or median. (Using Imputer Class)</li>
    </ul>
</span>

In [ ]:
df = spark.createDataFrame([
  (11.0, 66.0, 5.0),
  (12.0, 68.0, None),
  (1.0, None, 6.0),
  (2.0, 72.0, 7.0)], 
  ["hour", "temperature", "wind"]
)

In [ ]:
showDataFrame(df)

## Drop rows with missing values

In [ ]:
dropDF = df.dropna("any")

In [ ]:
showDataFrame(dropDF)

## Fill values with a placeholder

In [ ]:
plDF = df.na.fill({"temperature" : -1, "wind" : -1})

In [ ]:
showDataFrame(plDF)

## Using Imputer

In [ ]:
from pyspark.ml.feature import Imputer
timputer = Imputer(inputCols = ["temperature", "wind"], outputCols = ["tempFilled", "windFilled"], strategy = 'mean')
model = timputer.fit(df)
imputedDF = model.transform(df)

In [ ]:
showDataFrame(imputedDF)

<font color='gray'>
<h3>
<span style="font-family:verdana; font-size:1.2em;">
Duplicate Data<br>
</span>
</h3>
</font>
<span style="font-family:verdana; font-size:1.0em;">
Duplicate data might consists of multiple rows having same data, or they could have some fields that are not complete matches.<br>
Here is example of dropping duplicate data
</span>

In [ ]:
df = spark.createDataFrame([
  (15342, "Conor", "red"),
  (15342, "conor", "red"),
  (1234, "Conor", "red"),
  (12512, "Dorothy", "blue"),
  (5234, "Doug", "aqua")], 
  ["id", "name", "color"]
)

In [ ]:
showDataFrame(df)

In [ ]:
deDupedDF = df.dropDuplicates(["id", "color"])

In [ ]:
showDataFrame(deDupedDF)

<font color='gray'>
<h3>
<span style="font-family:verdana; font-size:1.2em;">
Pivot DataFrame<br>
</span>
</h3>
</font>
<span style="font-family:verdana; font-size:1.0em;">
PySaprk SQL provides a pivot() function that can rotate the data from one column to multiple columns. It is an aggregation where one of the grouping column values are transposed into individual columns with distinct data.<br>
</span>

In [ ]:
# define data
data = [("Banana",1000,"USA"), ("Carrots",1500,"USA"), ("Beans",1600,"USA"), \
      ("Orange",2000,"USA"),("Orange",2000,"USA"),("Banana",400,"China"), \
      ("Carrots",1200,"China"),("Beans",1500,"China"),("Orange",4000,"China"), \
      ("Banana",2000,"Canada"),("Carrots",2000,"Canada"),("Beans",2000,"Mexico")]

In [ ]:
# Create DataFrame
columns= ["Product","Amount","Country"]
df = spark.createDataFrame(data = data, schema = columns)

In [ ]:
df.printSchema()

In [ ]:
df.limit(20).toPandas()

In [ ]:
df.count()

In [ ]:
# Get list of unique country names
countries = [x.Country for x in df.select('Country').distinct().collect()]
countries

In [ ]:
# apply pivot function
pivotDF = df.groupBy("Product").pivot("Country", countries).sum("Amount")

In [ ]:
showDataFrame(pivotDF)

<font color='tomato'>
<h3>
<span style="font-family:verdana; font-size:1.2em;">
Exercise 1: Removing Duplicates<br>
</span>
</h3>
</font>
<span style="font-family:verdana; font-size:1.0em;">
Removing duplicates that do not completely match could be caused by user "entry/generated" error, schema errors or data corruption.
Perform the following steps:
    <ol>
        <li>Import csv file <b>people-with-dups.csv</b></li>
        <li>Show the dataframe, print schema and count</li>
        <li>Append Columns to filter duplicates, add following columns:</li>
        <ol>
            <li><b>lcFirstName:</b> lower case First Name</li>
            <li><b>lcLastName:</b> lower case Last Name</li>
            <li><b>lcMiddleName:</b> lower case Middle Name</li>
            <li><b>ssnNums:</b> remove '-' from the social security numbers</li>
            <li>Save this to a new DataFrame</li>
        </ol>
        <li>Drop Duplicates:</li>
        <ol>
            <li>Drop duplicates in all columns except (firstName, lastName, middleName, ssn)</li>
            <li>Drop the columns that were added in the step 3</li>
            <li>Save this to a new DataFrame</li>
            <li>Show DataFrame, print the counts</li>
        </ol>
        <li>Compare the counts before and after removing duplicates</li>
    </ol>
</span>

In [ ]:
if DATA_BRICKS == False:
    csvfile = "../datasets/people-with-dups.csv"
else:
    csvfile = "dbfs:/FileStore/datasets/people-with-dups.csv"